<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#2.-Training" data-toc-modified-id="2.-Training-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>2. Training</a></span><ul class="toc-item"><li><span><a href="#2-1.-Data-selection" data-toc-modified-id="2-1.-Data-selection-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>2-1. Data selection</a></span></li><li><span><a href="#2-2.-Training-Data-Preparation" data-toc-modified-id="2-2.-Training-Data-Preparation-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>2-2. Training Data Preparation</a></span></li><li><span><a href="#2-3.-Set-Training-Parameter" data-toc-modified-id="2-3.-Set-Training-Parameter-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>2-3. Set Training Parameter</a></span></li><li><span><a href="#2-4.-Training" data-toc-modified-id="2-4.-Training-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>2-4. Training</a></span></li><li><span><a href="#2-5.-Meta-Save" data-toc-modified-id="2-5.-Meta-Save-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>2-5. Meta Save</a></span></li></ul></li></ul></div>

In [1]:
import sys, os
import pandas as pd
import numpy as np
import setting
import pathSetting
sys.path.append("../../..")

from KETIToolDL.CLUSTTool.common import p1_integratedDataSaving as p1
from KETIToolDL.CLUSTTool.common import p2_dataSelection as p2
from KETIToolDL.CLUSTTool.common import p3_training as p3

import torch
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

c:\Users\korea\anaconda3\envs\torch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda is available.


# 2. Training 

## 2-1. Data selection

In [2]:
# 1. Train Data
# 1-1. 저장 Data 확인
DataMeta = p1.readJsonData(pathSetting.DataMetaPath)
dataList =  list(DataMeta.keys())

In [3]:
dataList

['ML_testX_ActionPatternData_Split3',
 'ML_testy_ActionPatternData_Split3',
 'ML_trainX_ActionPatternData_Split3',
 'ML_trainy_ActionPatternData_Split3',
 'trainX_ActionPatternData_Split3',
 'trainy_ActionPatternData_SplitNone',
 'testX_ActionPatternData_Split3',
 'testy_ActionPatternData_SplitNone']

In [20]:
dataList[4], dataList[5]

('trainX_ActionPatternData_Split3', 'trainy_ActionPatternData_SplitNone')

In [24]:
# 1-2. Select Train Data
## dataX
dataName_X = dataList[2]
dataSaveMode_X = DataMeta[dataName_X]["integrationInfo"]["DataSaveMode"]
windows = DataMeta[dataName_X]["integrationInfo"]["dataInfo"]["windows"]

## datay
dataName_y = dataList[3]
dataSaveMode_y = DataMeta[dataName_y]["integrationInfo"]["DataSaveMode"]

In [25]:
# 1-3. Read Train Data
## CSV 로 Local 에 데이터 저장되어 있을 경우
dataFolderName = "data_integrated_result"
current = os.getcwd()
dataFolderPath = os.path.join(current, dataFolderName)

dataX = p2.getSavedIntegratedData(dataSaveMode_X, dataName_X, dataFolderPath)
datay = p2.getSavedIntegratedData(dataSaveMode_y, dataName_y, dataFolderPath)
integration_freq_sec = DataMeta[dataName_X]["integrationInfo"]["integration_freq_sec"]

In [19]:
# data_X_reshape = pd.DataFrame(np.array(dataX).reshape(7352, 32, 9))

In [32]:
## MSH
# UCI HAR Data Read
# folderAddress = './data/'
# model_list = ["LSTM_cf","GRU_cf", "CNN_1D_cf","LSTM_FCNs_cf"]
# model_name = model_list[2]

# dataX, datay, _, _ = setting.getTrainDataFromFilesForClassification(folderAddress, model_name)

(7352, 9, 128)
(7352,)
(2947, 9, 128)
(2947,)
inputSize(train_x.shape[1]): 9
sequenceLenth (train_x.shape[2]): 128


## 2-2. Training Data Preparation

In [27]:
# 2. Training Data Preparation
# 2-1.
featureListX= list(dataX.columns)
featureListy= list(datay.columns)
target_col = 'value'

# 2-2.
cleanTrainDataParam = 'NoClean'#  Classification, Regression과 같이 X, y가 분리된 경우에는 현재 고정해서 사용해야함

# 2-3.
scalerParam="scale"
splitRatio = 0.8
scaleMethod='minmax'
modeX = 'windows_split'

scalerRootPath_X = os.path.join(pathSetting.scalerRootDir, dataName_X, cleanTrainDataParam)
scalerRootPath_y = None

# split train data into train/valiation data
# normalization
trainvalX, X_scalerFilePath = p3.getScaledData(scalerParam, scalerRootPath_X, dataX, scaleMethod)

# train data를 랜덤으로 test_size=split_ratio에 대하여 train/validation set으로 분할
split_ratio = 0.2
train_x, val_x, train_y, val_y = train_test_split(trainvalX, datay, test_size=split_ratio, shuffle=True)



# 2-4. Scale & Split Train/Validation
#scalerRootPath_X = os.path.join(pathSetting.scalerRootDir, dataName_X, cleanTrainDataParam)
#scalerRootPath_y = os.path.join(pathSetting.scalerRootDir, dataName_X, cleanTrainDataParam)
#train_x, val_x, X_scalerFilePath = p3.getTrainValData(dataX, featureListX, scalerRootPath_X, splitRatio, scalerParam, scaleMethod, modeX, windows)
#train_y, val_y, y_scalerFilePath = p3.getTrainValData(datay, featureListy, scalerRootPath_y, splitRatio, "NoScale")

['concat_emb1', 'concat_emb2', 'concat_emb3', 'concat_emb4', 'concat_emb5', 'concat_emb6', 'concat_emb7', 'concat_emb8', 'concat_emb9', 'concat_emb10', 'concat_emb11', 'concat_emb12', 'concat_emb13', 'concat_emb14', 'concat_emb15', 'concat_emb16', 'concat_emb17', 'concat_emb18', 'concat_emb19', 'concat_emb20', 'concat_emb21', 'concat_emb22', 'concat_emb23', 'concat_emb24', 'concat_emb25', 'concat_emb26', 'concat_emb27', 'concat_emb28', 'concat_emb29', 'concat_emb30', 'concat_emb31', 'concat_emb32']
Make New scaler File


## 2-3. Set Training Parameter

In [28]:
# 3. Set Training Parameter
# 3-1.
model_list = ["LSTM_cf","CNN_1D_cf", "FC_cf"]
model_method = model_list[0]

n_epochs = 100 # 학습 epoch 횟수, int(default: 1000, 범위: 1 이상)
batch_size = 64  # batch 크기, int(default: 16, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)
num_classes = 6 # class 개수

trainParameter = setting.modelConfig[model_method]
trainParameter['device']  = device
trainParameter['num_classes'] = num_classes

modelTags =["action", "sensor", "classification", "pattern", dataName_X, model_method]
trainDataType = "timeseries"
modelPurpose = "classification"

trainDataInfo = DataMeta[dataName_X]['integrationInfo']

# 3-2. 모델을 저장할 파일 패스를 생성
from KETIPreDataTransformation.general_transformation.dataScaler import encodeHashStyle
trainParameter_encode =  encodeHashStyle(str(trainParameter))

ModelName = dataName_X+"_"+model_method

trainDataPathList = [ModelName, dataName_X, trainParameter_encode]
modelFilePath = p3.getModelFilePath(trainDataPathList, model_method)


In [29]:
trainParameter

{'num_layers': 2,
 'hidden_size': 64,
 'dropout': 0.1,
 'bidirectional': True,
 'lr': 0.0001,
 'device': 'cuda',
 'num_classes': 6}

In [30]:
ModelName

'ML_trainX_ActionPatternData_Split3_LSTM_cf'

## 2-4. Training

In [31]:
trainParameter

{'num_layers': 2,
 'hidden_size': 64,
 'dropout': 0.1,
 'bidirectional': True,
 'lr': 0.0001,
 'device': 'cuda',
 'num_classes': 6}

In [26]:
# 4. Training
from KETIToolDL.TrainTool.Classification.trainer import ClassificationML as CML

cml = CML(model_method, trainParameter)
cml.processInputData(train_x, train_y, val_x, val_y, batch_size, windows)
model = cml.getModel()
best_model, timeElapsed = cml.trainModel(model, modelFilePath, n_epochs)

Start training model

Epoch 1/100
train Loss: 1.7728 Acc: 0.3719
val Loss: 1.7467 Acc: 0.5316

Epoch 10/100
train Loss: 0.0145 Acc: 1.0000
val Loss: 0.0125 Acc: 1.0000

Epoch 20/100
train Loss: 0.0025 Acc: 1.0000
val Loss: 0.0024 Acc: 1.0000

Epoch 30/100
train Loss: 0.0010 Acc: 1.0000
val Loss: 0.0009 Acc: 1.0000

Epoch 40/100
train Loss: 0.0005 Acc: 1.0000
val Loss: 0.0005 Acc: 1.0000

Epoch 50/100
train Loss: 0.0003 Acc: 1.0000
val Loss: 0.0002 Acc: 1.0000

Epoch 60/100
train Loss: 0.0001 Acc: 1.0000
val Loss: 0.0001 Acc: 1.0000

Epoch 70/100
train Loss: 0.0001 Acc: 1.0000
val Loss: 0.0001 Acc: 1.0000

Epoch 80/100
train Loss: 0.0001 Acc: 1.0000
val Loss: 0.0000 Acc: 1.0000

Epoch 90/100
train Loss: 0.0000 Acc: 1.0000
val Loss: 0.0000 Acc: 1.0000

Epoch 100/100
train Loss: 0.0000 Acc: 1.0000
val Loss: 0.0000 Acc: 1.0000

Training complete in 0m 24s
Best val Acc: 1.000000


## 2-5. Meta Save

In [29]:
# 5. Meta Save
# 5-1. model name 확인
ModelName

'ML_trainX_ActionPatternData_Split3_LSTM_cf'

In [30]:
# 5-2. update MetaFile
modelInfoMeta ={
    "author":"KETI",
    "modelName": ModelName,
    "modelVersion":0.1,
    "trainDataInfo":trainDataInfo,
    "featureList":featureListX,
    "target":featureListy,
    "trainDataType":trainDataType,
    "modelPurpose":modelPurpose,
    "model_method":model_method,
    "modelTags":modelTags,
    "cleanTrainDataParam":cleanTrainDataParam,
    "NaNProcessingParam":{},
    "trainDataName":[dataName_X,dataName_y],
    "trainParameter":cml.parameter,
    "scalerParam":scalerParam,
    "files":{
        "modelFile":{
            "fileName":"model.pth",
            "filePath":modelFilePath
        },
        "XScalerFile":{
            "fileName":"scaler.pkl",
            "filePath":X_scalerFilePath       
        },
        "yScalerFile":{
            "fileName":"scaler.pkl",
            "filePath":scalerRootPath_y       
        }
    },
    "trainParameter":trainParameter,
    "n_epochs":n_epochs,
    "timeElapsed" : timeElapsed
}
modelInfoMeta = p3.updateModelMetaData(ModelName, modelInfoMeta, pathSetting.trainModelMetaFilePath)

In [31]:
modelInfoMeta

{'author': 'KETI',
 'modelName': 'ML_trainX_ActionPatternData_Split3_LSTM_cf',
 'modelVersion': 0.1,
 'trainDataInfo': {'startTime': '2022-01-01',
  'endTime': '2022-01-01 08:10:04',
  'dataInfo': {'windows': 1,
   'dataSize': '235264',
   'timeElapsed': '0:00:52.012849'},
  'processParam': {'refine_param': {'removeDuplication': {'flag': False},
    'staticFrequency': {'flag': False, 'frequency': None}},
   'outlier_param': {'certainErrorToNaN': {'flag': False},
    'unCertainErrorToNaN': {'flag': False, 'param': {}}},
   'imputation_param': {'serialImputation': {'flag': False,
     'imputation_method': [],
     'totalNonNanRatio': 80}}},
  'integration_freq_sec': 4,
  'cleanParam': 'NoClean',
  'DataSaveMode': 'CSV'},
 'featureList': ['concat_emb1',
  'concat_emb2',
  'concat_emb3',
  'concat_emb4',
  'concat_emb5',
  'concat_emb6',
  'concat_emb7',
  'concat_emb8',
  'concat_emb9',
  'concat_emb10',
  'concat_emb11',
  'concat_emb12',
  'concat_emb13',
  'concat_emb14',
  'concat_emb